## Math model scripts


### first we create a generating function to keep track of the probability that the number of approvals = k
### the generating function is in the form:
$g(x) = p_a(x)^1 + p_r(x)^-1 + p_i(x)^0$
for a single approval

for n many agents sending an approval, the generating function looks like:
$g(x)^n = (p_a(x)^1 + p_r(x)^-1 + p_i(x)^0)^n$

In [1]:
import sympy as sp
#we will manipulate the generating function symbolically to compute the probability of success in a single depth level for a single parent
import math
import numpy as np

In [ ]:
def generating_function(x:sp.Symbol, p_a:sp.Symbol, p_r:sp.Symbol, p_i:sp.Symbol, n:sp.Symbol)->sp.Expr:

    g = (p_a*(x**1) + p_r*(x**-1) + p_i*(x**0))**n
    return g

In [ ]:
x, n = sp.symbols('x, n')
p_a, p_r, p_i = sp.symbols('p_a, p_r, p_i')

g = generating_function(x, p_a, p_r, p_i, n)

In [ ]:
#here we will sub into n the number of neighbours a parent has at depth d
gn = g.subs(n, 2)
expanded_gn = sp.expand(gn)

In [ ]:
#TODO: make this into a function to return coeff_dict
# Dictionary to hold coefficients by power
coeff_dict = {}

# Break the expression into terms and analyze powers
for term in expanded_gn.as_ordered_terms():
    term = sp.expand(term)
    coeff, power = term.as_coeff_exponent(x)
    coeff_dict[power] = coeff_dict.get(power, 0) + coeff

# Sort and display coefficients by power of x
for power in sorted(coeff_dict):
    print(f"Coefficient of x^{power}: {coeff_dict[power]}")

In [ ]:
#TODO: also make into a function to return prob success per depth level
#range(np.ceil(n_d*t), n_d +1)
prob_success_at_level_d = 0
for exponent in range(-2, 3):
    #print('coefficient', coeff_dict[exponent])
    print('subbed in coefficient', coeff_dict[exponent].subs({p_a: 0.5, p_r: 0.3, p_i: 0.2, x: 1}))
    prob_success_at_level_d += coeff_dict[exponent].subs({p_a: 0.5, p_r: 0.3, p_i: 0.2, x: 1})
print('SUCCESS AT DEPTH LEVEL D', prob_success_at_level_d)


In [ ]:
def binomial_probability(p_a, n, k):
        n_choice_k = math.factorial(n)/(math.factorial(k) * math.factorial(n-k))
        binomial_prob = n_choice_k * ((p_a ** k) * ((1-p_a) ** (n-k)))
        return binomial_prob

In [ ]:
def probability_of_success(p_s:float, N:int, t:float)->float:
    """p_s: probability of a single parent receiving enough approvals from their n_d children
    N: number of parents in that depth level
    t: threshold
    
    returns: probability of enough PARENTS succeeding at depth level d"""
    k = int(np.ceil(N*t))

    for i in range(N, k):

        # Calculate the binomial probability
        prob_sucess_depth = 0

        #summing over all the possible valid outcomes. (ie: from getting k approvals, to k+1, k+2, ... n)
        for i in range(k, N+1):
            #print(i)
            binomial_prob= binomial_probability(p_s, n, i)
            prob_sucess_depth += binomial_prob
    return prob_sucess_depth


In [ ]:
def prob_TCA_True(height, branching_factor, threshold):
    total_prob_sucess = 1
    for d in range(height-1, -1, -1):
        n = branching_factor[d]
        k = branching_factor[d] * threshold
